In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers
!pip install IPython
!pip install tqdm
!pip install torch
!pip install kobert_transformers
!pip install kogpt2_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00


In [4]:
%run /content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot/model/koelectra.py
%run /content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot/dataloader/wellness.py
%run /content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot/model//kobert.py

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


AttributeError: 'KoBertTokenizer' object has no attribute 'token2idx'

ModuleNotFoundError: No module named 'model'

In [5]:
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from tqdm import tqdm

import torch
from transformers import (
  AdamW,
  ElectraConfig,
  ElectraTokenizer
)
from torch.utils.data import dataloader
from kobert_transformers import get_tokenizer


%cd /content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot
from dataloader.wellness import WellnessTextClassificationDataset
from model.koelectra import koElectraForSequenceClassification
# from model.kobert import KoBERTforSequenceClassfication


logger = logging.getLogger(__name__)

/content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot


In [6]:
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from tqdm import tqdm

import torch
from transformers import (
  AdamW,
  ElectraConfig,
  ElectraTokenizer
)
from torch.utils.data import dataloader
from kobert_transformers import get_tokenizer


%cd /content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot
from dataloader.wellness import WellnessTextClassificationDataset
from model.koelectra import koElectraForSequenceClassification
# from model.kobert import KoBERTforSequenceClassfication


logger = logging.getLogger(__name__)


# MODEL_CLASSES ={
#   "koelectra": (ElectraConfig, koElectraForSequenceClassification, ElectraTokenizer),
#   "kobert": (KoBERTforSequenceClassfication)
# }
# CHECK_POINT ={
#   "koelectra": f"{root_path}/checkpoint/koelectra-wellnesee-text-classification.pth",
#   "kobert": f"{root_path}/checkpoint/kobert-wellnese.pth"
# }
root_path = '/content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot'

MODEL_CLASSES ={
  "koelectra": (ElectraConfig, koElectraForSequenceClassification, ElectraTokenizer)
}
CHECK_POINT ={
  "koelectra": f"{root_path}/checkpoint/koelectra-wellnesee-text-classification-25.pth-train"
}
def get_model_and_tokenizer(model_name, device):
  save_ckpt_path = CHECK_POINT['koelectra']

  model_name== "koelectra"
  model_name_or_path = "monologg/koelectra-base-discriminator"

  tokenizer = ElectraTokenizer.from_pretrained(model_name_or_path)
  electra_config = ElectraConfig.from_pretrained(model_name_or_path)
  model = koElectraForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path,
                                                              config=electra_config,
                                                              num_labels=359)

  if os.path.isfile(save_ckpt_path):
      checkpoint = torch.load(save_ckpt_path, map_location=device)
      pre_epoch = checkpoint['epoch']
      # pre_loss = checkpoint['loss']
      model.load_state_dict(checkpoint['model_state_dict'])

      print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}")

  return model, tokenizer

def get_model_input(data):
  if model_name =='kobert':
    return data
  elif model_name== "koelectra":
    return {'input_ids': data['input_ids'],
              'attention_mask': data['attention_mask'],
              'labels': data['labels']
              }

def evaluate(model_name, device, batch_size, data_path):

  model, tokenizer = get_model_and_tokenizer(model_name, device)
  model.to(device)

  # WellnessTextClassificationDataset 데이터 로더
  eval_dataset = WellnessTextClassificationDataset(file_path=data_path,device=device, tokenizer=tokenizer)
  eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, shuffle=True)

  logger.info("***** Running evaluation on %s dataset *****")
  logger.info("  Num examples = %d", len(eval_dataset))
  logger.info("  Batch size = %d", batch_size)

  loss = 0
  acc = 0


  # model.eval()
  for data in tqdm(eval_dataloader, desc="Evaluating"):
    with torch.no_grad():
      inputs = get_model_input(data)
      outputs = model(**inputs)
      loss += outputs[0]
      logit = outputs[1]
      acc += (logit.argmax(1)==inputs['labels']).sum().item()

  return loss / len(eval_dataset), acc / len(eval_dataset)

if __name__ == '__main__':

  root_path = "/content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot"
  data_path = f"{root_path}/data/wellness_data_for_text_classification_test.txt"
  save_ckpt_path = f"{root_path}/checkpoint/koelectra-wellnesee-text-classification-25-train.pth"
  model_name_or_path = "monologg/koelectra-base-discriminator"

  n_epoch = 50  # Num of Epoch
  batch_size = 16  # 배치 사이즈
  ctx = "cuda" if torch.cuda.is_available() else "cpu"
  device = torch.device(ctx)
  model_names=["koelectra"]
  for model_name in model_names:
    eval_loss, eval_acc = evaluate(model_name, device, batch_size, data_path)
    print(f'\tLoss: {eval_loss:.4f}(valid)\t|\tAcc: {eval_acc * 100:.1f}%(valid)')

/content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/279k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of koElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating: 100%|██████████| 66/66 [38:53<00:00, 35.35s/it]

	Loss: 0.3727(valid)	|	Acc: 0.1%(valid)
